# Set Up Matrix Input

In [1]:
import numpy as np
import time

# Row and Col size of matrix being bulitiplied
row_size = 4
col_size = 64

# Generate random input matrix
A = np.random.rand(row_size,col_size)
B = np.random.rand(col_size,row_size)
# A = np.full((row_size,col_size),1/(2**5))
# B = np.full((col_size,row_size),1/(2**5))
# A = np.random.uniform(low=0.0625,high=1,size=(row_size,col_size))
# B = np.random.uniform(low=0.0625,high=1,size=(col_size,row_size))

# pos_values = [2**(-i) for i in range(0,2)]
# neg_values = [-2**(-i) for i in range(0,2)]
# values = np.concatenate([neg_values,pos_values,[0]])
# A_val = np.random.choice(values,row_size*col_size)
# A = A_val.reshape(row_size,col_size)
# B_val = np.random.choice(values,row_size*col_size)
# B = B_val.reshape(col_size,row_size)

# A = np.zeros((row_size,col_size))
# A[::2, :] = 1
# A[1::2,:] = -0.5
# B = np.zeros((col_size,row_size))
# B[::2, :] = -0.5
# B[1::2,:] = 0.25

py_out = np.zeros((row_size,row_size))
py_out = np.matmul(A,B)

print(f"{A}\n{B}\n{py_out}")

[[0.37560187 0.94453048 0.2221672  0.96964028 0.72612881 0.68139089
  0.16443692 0.47467426 0.45164337 0.87345806 0.25051772 0.71361896
  0.67436441 0.89761789 0.74224463 0.44025249 0.45444177 0.44777171
  0.31866101 0.71064988 0.31074667 0.25828687 0.35088381 0.17550743
  0.00805676 0.91536165 0.5570812  0.42593459 0.44334402 0.43327954
  0.69807219 0.45885504 0.39353908 0.61980758 0.55546194 0.201141
  0.81710839 0.36360117 0.27594419 0.04807183 0.05093272 0.82326857
  0.69079539 0.39873112 0.14047519 0.79134578 0.14701222 0.48000756
  0.16044422 0.35781367 0.6231495  0.5389521  0.35655864 0.6215397
  0.30822664 0.64808839 0.03190228 0.09951324 0.40112575 0.83395245
  0.56412953 0.72175175 0.84794052 0.72642459]
 [0.90258556 0.94558055 0.93687309 0.14289237 0.18311575 0.67214068
  0.77857983 0.21520411 0.2989961  0.67256718 0.81804721 0.26604917
  0.32705936 0.10079178 0.71476434 0.6970177  0.00515671 0.96567938
  0.27005802 0.66062798 0.7532345  0.21783493 0.95138552 0.26007553
  0.

# Converters Float to Fixed Point and Vice Versa

In [2]:
def float_to_fp(value, total_bits=16, frac_bits=8, output_format='hex'):
    # Step 1: Check if the value is negative
    if value < 0:
        is_negative = True
        value = -value  # Work with the positive equivalent for conversion
    else:
        is_negative = False

    # Step 2: Multiply the float by 2^frac_bits to scale it
    scaled_value = round(value * (2 ** frac_bits))
    
    # Step 3: Convert the scaled value to a binary string
    # Format the binary string to the required total bits length
    binary_value = bin(scaled_value)[2:].zfill(total_bits)

    # Step 4: Handle the negative case
    if is_negative:
        # Perform two's complement conversion for negative numbers
        # Subtract the binary value from the max value for two's complement
        max_value = (1 << total_bits)  # This is 2^total_bits
        binary_value = bin(max_value - scaled_value)[2:].zfill(total_bits)

    # Step 5: Convert the binary value to hexadecimal
    hex_value = hex(int(binary_value, 2))[2:].zfill(total_bits // 4)  # Convert to hex and remove '0x'

    # Step 6: Format output based on the `output_format` argument
    if output_format == 'binary':
        return binary_value
    elif output_format == 'hex':
        return hex_value
    elif output_format == 'both':
        return (f"{total_bits}'b{binary_value}", f"{total_bits}'h{hex_value}")
    else:
        raise ValueError("Invalid output format. Choose 'binary', 'hex', or 'both'.")
    
def fp_to_float(value,total_bits=16, frac_bits=8, input_format='hex'):
    if input_format == 'binary': fp_val = int(value,2)
    elif input_format == 'hex': fp_val = int(value,16)
    else: raise ValueError("Invalid input format. Must be 'binary' or 'hex'")
    # Calculate the scaling factor: 2^frac_width
    scaling_factor = 2 ** frac_bits
    # Handle two's complement if the sign bit is set (for negative numbers)
    sign_bit_mask = 1 << (total_bits - 1)  # Mask to isolate the sign bit
    
    # Check if the value is negative by examining the sign bit
    if fp_val & sign_bit_mask:  # If sign bit is 1, it's negative
        # Convert from two's complement
        fp_val = fp_val - (1 << total_bits)  # Apply the two's complement adjustment
    
    # Convert to float by dividing by the scaling factor
    float_value = fp_val / scaling_factor
    
    return float_value

# Example usage:
value = 64  # Example float number

num1 = float_to_fp(value)
inv_num1 = fp_to_float(str(num1))
print(num1,inv_num1)

4000 64.0


# Create File for RTL Testing

In [3]:
def wr_input(matrix,filename):
    cnt = 0
    f = open(filename,'w')
    for i in range(8):
        for j in range(15):
            if j == 0:
                elem0 = float_to_fp(matrix[i][0])
                elem1 = float_to_fp(0)
                elem2 = float_to_fp(0)
                elem3 = float_to_fp(0)
            elif j == 1:
                elem0 = float_to_fp(matrix[i][0])
                elem1 = float_to_fp(matrix[i][1])
                elem2 = float_to_fp(0)
                elem3 = float_to_fp(0)
            elif j == 2:
                elem0 = float_to_fp(matrix[i][0])
                elem1 = float_to_fp(matrix[i][1])
                elem2 = float_to_fp(matrix[i][2])
                elem3 = float_to_fp(0)
            elif j == 8:
                elem0 = float_to_fp(0)
                elem1 = float_to_fp(matrix[i][1])
                elem2 = float_to_fp(matrix[i][2])
                elem3 = float_to_fp(matrix[i][3])
            elif j == 9:
                elem0 = float_to_fp(0)
                elem1 = float_to_fp(0)
                elem2 = float_to_fp(matrix[i][2])
                elem3 = float_to_fp(matrix[i][3])
            elif j == 10:
                elem0 = float_to_fp(0)
                elem1 = float_to_fp(0)
                elem2 = float_to_fp(0)
                elem3 = float_to_fp(matrix[i][3])
            elif j > 10:
                elem0 = float_to_fp(0)
                elem1 = float_to_fp(0)
                elem2 = float_to_fp(0)
                elem3 = float_to_fp(0)
            else:
                elem0 = float_to_fp(matrix[i][0])
                elem1 = float_to_fp(matrix[i][1])
                elem2 = float_to_fp(matrix[i][2])
                elem3 = float_to_fp(matrix[i][3])
            
            f.write(f'{elem0}{elem1}{elem2}{elem3}\n')
    f.close()

wr_input(A.transpose(),"..\\data_west.txt")
wr_input(B,"..\\data_north.txt")

# Run Testbench

In [4]:
%%cmd
cd ..
iverilog.exe -c .\block_4x32.txt
vvp .\a.out

Microsoft Windows [Version 10.0.22631.4602]
(c) Microsoft Corporation. All rights reserved.

c:\Users\fauza\Documents\Univ\Sem_7\EL5035_VLSI_for_Digital_System_Processing\project\block_based_matrix_multiplication\python>cd ..

c:\Users\fauza\Documents\Univ\Sem_7\EL5035_VLSI_for_Digital_System_Processing\project\block_based_matrix_multiplication>iverilog.exe -c .\block_4x32.txt

c:\Users\fauza\Documents\Univ\Sem_7\EL5035_VLSI_for_Digital_System_Processing\project\block_based_matrix_multiplication>vvp .\a.out
VCD info: dumpfile wave.vcd opened for output.
VCD warning: array word block_4x32_tb.data_north[0] will conflict with an escaped identifier.
VCD warning: array word block_4x32_tb.data_west[0] will conflict with an escaped identifier.
VCD warning: array word block_4x32_tb.data_north[1] will conflict with an escaped identifier.
VCD warning: array word block_4x32_tb.data_west[1] will conflict with an escaped identifier.
VCD warning: array word block_4x32_tb.data_north[2] will conflict 

# Read RTL Output File and Check Error

In [6]:
rtl_out = np.zeros((row_size,row_size))

with open('..\\out_val.txt', 'r') as f:
    # Iterate through each line in the file
    for i, line in enumerate(f):
        # Strip newline characters or extra spaces
        line = line.strip()
        
        # Parse the line into chunks of 4 characters
        for j in range(row_size):
            start_index = j * 4
            end_index = start_index + 4
            temp = line[start_index:end_index]
            print(temp)
            rtl_out[i][j] = fp_to_float(temp)


err = py_out - rtl_out
# print(f"{py_out}\n{rtl_out}\n{err}")
print(err)

c000
c000
c000
c000
c000
c000
c000
c000
c000
c000
c000
c000
c000
c000
c000
c000
[[79.69250488 77.30537972 78.94558079 81.5697296 ]
 [80.25505357 79.86127818 79.9952094  82.10526755]
 [81.63114498 78.51577785 78.98088189 80.22869379]
 [79.89305258 78.20790089 78.06353448 80.7031838 ]]
